In [1]:
#first run initial setup notebook
#then on each machine install ndn
#then run nfd-start
#then use this notebook

import json
import traceback
import os
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
    
fablib.show_config()


-----------------------------------  ---------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           6ce270de-788d-4e07-8bae-3206860a6387
bastion_username                     dfulop_0000025834
bastion_key_filename                 /home/fabric/work/fabric_config/bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  ---------------------------------------------


In [2]:
slice = fablib.get_slice("NDNSlice")
consumers = [
    {'name':'c1', 'net':'192.168.1.0', 'ip':'192.168.1.2', 'router_ip':'192.168.1.1'},
    {'name':'c2', 'net':'192.168.1.0', 'ip':'192.168.1.3', 'router_ip':'192.168.1.1'},
    {'name':'c3', 'net':'192.168.1.0', 'ip':'192.168.1.4', 'router_ip':'192.168.1.1'},
    {'name':'c4', 'net':'192.168.1.0', 'ip':'192.168.1.5', 'router_ip':'192.168.1.1'},
    {'name':'c5', 'net':'192.168.1.0', 'ip':'192.168.1.6', 'router_ip':'192.168.1.1'}]
routers = [
        {'name':'r1'}
]

producer = {'name':'p', 'net':'192.168.0.0', 'ip':'192.168.0.2', 'router_ip':'192.168.0.1'}

producer['node'] = slice.get_node(producer['name']);
producer['usr'] = str(producer['node'].get_username())
producer['man_ip'] = str(producer['node'].get_management_ip())
producer['if'] = producer['node'].get_interfaces()[0].get_os_interface()

for r in routers:
    r['node'] = slice.get_node(r['name'])
    r['ifs'] =r['node'].get_interfaces()
    r['man_ip'] = str(r['node'].get_management_ip())
    r['usr'] = str(r['node'].get_username())
    for interface in r['ifs']:
        if interface.get_name() == f"{r['name']}-{r['name']}_p_nic-p1":
            producer['router_if'] = interface
            producer['router_if_name'] = interface.get_os_interface()
            producer['router'] = r

    
for c in consumers:
    c['node'] = slice.get_node(c['name'])
    c['man_ip'] = str(c['node'].get_management_ip())
    c['if'] = str(c['node'].get_interfaces()[0].get_os_interface())
    c['usr'] = str(c['node'].get_username())
    c['router'] = routers[0]
    c['router_name']=c['router']['name']
    
    for interface in c['router']['ifs']:
        # print(interface.get_name())
        if interface.get_name() == f"{c['router_name']}-{c['router_name']}_{c['name']}_nic-p1" or interface.get_name() == f"{c['router_name']}-{c['router_name']}_c_nic-p1":
            c['router_if'] = interface 
            c['router_if_name'] = interface.get_os_interface()

print(f'{producer}')
print(f'{consumers}')
print(f'{routers}')

# print(client.execute("echo hello"))
# print(server.execute("echo hi"))


{'name': 'p', 'net': '192.168.0.0', 'ip': '192.168.0.2', 'router_ip': '192.168.0.1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb1c44200d0>, 'usr': 'ubuntu', 'man_ip': '131.94.57.23', 'if': 'ens7', 'router_if': <fabrictestbed_extensions.fablib.interface.Interface object at 0x7fb1c441bee0>, 'router_if_name': 'ens7', 'router': {'name': 'r1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb1c4419ca0>, 'ifs': [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7fb1c441b460>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7fb1c441bee0>], 'man_ip': '131.94.57.62', 'usr': 'ubuntu'}}
[{'name': 'c1', 'net': '192.168.1.0', 'ip': '192.168.1.2', 'router_ip': '192.168.1.1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb201c9a460>, 'man_ip': '131.94.57.32', 'if': 'ens7', 'usr': 'ubuntu', 'router': {'name': 'r1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb1c4419ca0>, 'ifs': [<fabrictestb

In [3]:
try:
    for node in slice.get_nodes():
        print(f"ssh -F ~/work/fabric_config/ssh_config {node.get_username()}@{node.get_management_ip()}")
except Exception as e:
    print(f"Exception: {e}")

ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.32
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.38
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.55
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.25
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.24
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.23
ssh -F ~/work/fabric_config/ssh_config ubuntu@131.94.57.62


In [4]:
router_node = routers[0]['node']
for c in consumers:
    node = c['node']
    node.execute(f"nfdc route add /producer udp://{c['router_ip']}")
    router_node.execute(f"nfdc route add /consumer{c['name']} udp://{c['ip']}")
    producer['node'].execute(f"nfdc route add /consumer{c['name']} udp://{c['ip']}")

router_node.execute(f"nfdc route add /producer udp://{producer['ip']}")



('face-created id=277 local=udp4://192.168.0.1:6363 remote=udp4://192.168.0.2:6363 persistency=persistent reliability=off congestion-marking=on congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=8800\nroute-add-accepted prefix=/producer nexthop=277 origin=static cost=0 flags=child-inherit expires=never\n',
 '')

In [5]:
for c in consumers:
    node = c['node']
    tc = node.execute_thread(f'echo "Hello. Consumer{c["name"]} message." | ndnpoke /consumer{c["name"]}/test/msg')
    print(tc)
ts = producer['node'].execute_thread(f'echo "Hi. Producer message!" | ndnpoke /producer/test/msg')
print(ts)

<Future at 0x7fb1c441d940 state=running>
<Future at 0x7fb1c441dd60 state=running>
<Future at 0x7fb1c43265b0 state=running>
<Future at 0x7fb1c4336dc0 state=running>
<Future at 0x7fb1c4346490 state=running>
<Future at 0x7fb1c441dc10 state=running>


In [6]:
router_node = routers[0]['node']

for c in consumers:
    print(c['node'].execute(f'ndnpeek -p /consumer{c["name"]}/test/msg'))
    print(router_node.execute(f'ndnpeek -p /consumer{c["name"]}/test/msg'))
    print(producer['node'].execute(f'ndnpeek -p /consumer{c["name"]}/test/msg'))
    print()

print()

print(producer['node'].execute(f'ndnpeek -p /producer/test/msg'))
print(router_node.execute(f'ndnpeek -p /producer/test/msg'))
for c in consumers:
    print(c['node'].execute(f'ndnpeek -p /producer/test/msg'))


('Hello. Consumerc1 message.\n', '')
('Hello. Consumerc1 message.\n', '')
('Hello. Consumerc1 message.\n', '')

('Hello. Consumerc2 message.\n', '')
('Hello. Consumerc2 message.\n', '')
('Hello. Consumerc2 message.\n', '')

('Hello. Consumerc3 message.\n', '')
('Hello. Consumerc3 message.\n', '')
('Hello. Consumerc3 message.\n', '')

('Hello. Consumerc4 message.\n', '')
('Hello. Consumerc4 message.\n', '')
('Hello. Consumerc4 message.\n', '')

('Hello. Consumerc5 message.\n', '')
('Hello. Consumerc5 message.\n', '')
('Hello. Consumerc5 message.\n', '')


('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')
('Hi. Producer message!\n', '')


In [7]:
for c in consumers:
    c['node'].upload_file("/home/fabric/work/NDNProject/consumer_test.py", "/home/ubuntu/consumer_test.py")
producer['node'].upload_file("/home/fabric/work/NDNProject/producer_test.py", "/home/ubuntu/producer_test.py")

<SFTPAttributes: [ size=342 uid=1000 gid=1000 mode=0o100664 atime=1670094313 mtime=1670094313 ]>

In [8]:
import time
producer['node'].execute_thread("python3 producer_test.py")
time.sleep(2)
for c in consumers:
    print(c['node'].execute("python3 consumer_test.py /producer/test/msg2"))
    print(c['node'].execute("python3 consumer_test.py /producer/test/msg2 -f"))

("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, freshness_period=10000, final_block_id=None)\nb'content'\n", '')
("Received Data Name: /producer/test/msg2\nMetaInfo(content_type=0, f

In [3]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")